In [1]:
import pandas as pd
import numpy as np
import torch
import tqdm
import os
import random
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

2021-10-24 09:53:07.088063: I tensorflow/stream_executor/platform/default/dso_loader.cc:54] Successfully opened dynamic library libcudart.so.11.0


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="4"
tf_device='/gpu:0'

In [3]:
tokenizer = BertTokenizer.from_pretrained("indolem/indobert-base-uncased")
config = BertConfig.from_pretrained("indolem/indobert-base-uncased")
config.num_labels = 3
model = BertForSequenceClassification.from_pretrained("indolem/indobert-base-uncased", config=config)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indober

In [4]:
class DocumentSentimentDataset(Dataset):
    # Static constant variable (We need to have this part to comply with IndoNLU standard)
    LABEL2INDEX = {'negative': 0, 'neutral': 1, 'positive': 2} # Label string to index
    INDEX2LABEL = {0:'negative', 1:'neutral', 2: 'positive'} # Index to label string
    NUM_LABELS = 3 # Number of label
   
    def load_dataset(self, path):
        df = pd.read_csv(path, sep='\t', header=None) # Read tsv file with pandas
        df.columns = ['text','sentiment'] # Rename the columns
        df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Convert string label into index
        return df
   
    def __init__(self, dataset_path, tokenizer, *args, **kwargs):
        self.data = self.load_dataset(dataset_path) # Load the tsv file

        # Assign the tokenizer for tokenization
        # here we use subword tokenizer from HuggingFace
        self.tokenizer = tokenizer
    
    def __getitem__(self, index):
        data = self.data.loc[index,:] # Taking data from a specific row from Pandas
        text, sentiment = data['text'], data['sentiment'] # Take text and sentiment from the row
        subwords = self.tokenizer.encode(text) # Tokenize the text with tokenizer
        return np.array(subwords), np.array(sentiment), data['text']
   
    def __len__(self):
        return len(self.data)  # Return the length of the dataset

In [5]:
class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.max_seq_len = max_seq_len # Assign max limit of the sequence length
        self.collate_fn = self._collate_fn # Assign the collate_fn function with our function
       
    def _collate_fn(self, batch):
        batch_size = len(batch) # Take the batch size
        max_seq_len = max(map(lambda x: len(x[0]), batch)) # Find maximum sequence length from the batch 
        max_seq_len = min(self.max_seq_len, max_seq_len) # Compare with our defined limit
       
    # Create buffer for subword, mask, and sentiment labels, initialize all with 0
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
       
    # Fill all of the buffer
        seq_list = []
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
            
            seq_list.append(raw_seq)
           
    # Return the subword, mask, and sentiment data
        return subword_batch, mask_batch, sentiment_batch, seq_list

In [6]:
def forward_sequence_classification(model, batch_data, i2w, is_test=False, device = 'cuda', **kwargs):
    # Unpack batch data
    if len(batch_data) == 3:
        (subword_batch, mask_batch, label_batch) = batch_data
        token_type_batch = None
    elif len(batch_data) == 4:
        (subword_batch, mask_batch, token_type_batch, label_batch) = batch_data
    
    # Prepare input & label
    subword_batch = torch.LongTensor(subword_batch)
    mask_batch = torch.FloatTensor(mask_batch)
    token_type_batch = torch.LongTensor(token_type_batch) if token_type_batch is not None else None
    label_batch = torch.LongTensor(label_batch)
            
    if device == "cuda":
        subword_batch = subword_batch.cuda()
        mask_batch = mask_batch.cuda()
        token_type_batch = token_type_batch.cuda() if token_type_batch is not None else None
        label_batch = label_batch.cuda()
    
    # Forward model
    outputs = model(subword_batch, attention_mask=mask_batch, token_type_ids=token_type_batch, labels=label_batch)
    print(outputs)
    loss, logits = outputs[:2]
    
    # generate prediction & label list
    list_hyp = []
    list_label = []
    hyp = torch.topk(logits, 1)[1]
    for j in range(len(hyp)):
        list_hyp.append(i2w[hyp[j].item()])
        list_label.append(i2w[label_batch[j][0].item()])
    return loss, list_hyp, list_label

In [7]:
def document_sentiment_metrics_fn(list_hyp, list_label):
    metrics = {}
    metrics["ACC"] = accuracy_score(list_label, list_hyp)
    metrics["F1"] = f1_score(list_label, list_hyp, average='macro')
    metrics["REC"] = recall_score(list_label, list_hyp, average='macro')
    metrics["PRE"] = precision_score(list_label, list_hyp, average='macro')
    return metrics

In [8]:
import itertools
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(241021)

In [9]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


In [10]:
text =  'satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
print(subwords)
logits = model(subwords)[0]
print(logits)
print(torch.topk(logits, k=1))
label = torch.topk(logits, k=1)[1].squeeze().item()
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

tensor([[   3, 1713, 1713, 2254, 5458, 2477,   16, 1854, 1854, 1614, 5458, 3095,
           16, 2139,   17, 2139, 5458, 4962, 6962,   16, 1713, 1854, 2139, 5458,
         4398,    4]])
tensor([[-0.0717, -0.1426, -0.1880]], grad_fn=<AddmmBackward0>)
torch.return_types.topk(
values=tensor([[-0.0717]], grad_fn=<TopkBackward0>),
indices=tensor([[0]]))
Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : negative (35.43781280517578%)


In [11]:
print(f'Text: {text} | Label : {i2w[label]} ({torch.nn.functional.softmax(logits, dim=-1).squeeze()[label] * 100}%)')

Text: satu satu aku sayang ibu, dua dua juga sayang ayah, tiga-tiga sayang adik kakak, satu dua tiga sayang semuanya | Label : negative (35.43781280517578%)


In [12]:
train = DocumentSentimentDataset('dataset/train_preprocess.tsv', tokenizer)
valid = DocumentSentimentDataset('dataset/valid_preprocess.tsv', tokenizer)
train_loader = DocumentSentimentDataLoader(dataset=train, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)
valid_loader = DocumentSentimentDataLoader(dataset=valid, max_seq_len=100, batch_size=32, num_workers=1, shuffle=True)

In [13]:
torch.cuda.empty_cache()

In [14]:
def count_param(module, trainable=True):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
count_param(model)

110560515

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-6)
model = model.cuda()

In [16]:
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []

    train_pbar = tqdm.tqdm(iter(train_loader), leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')

        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss

        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label

        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))

    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))

(Epoch 1) TRAIN LOSS:1.1248 LR:0.00000300:   4%|█▋                                       | 1/25 [00:00<00:07,  3.33it/s]

SequenceClassifierOutput(loss=tensor(1.1248, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 1.6925e-01,  1.9000e-01, -4.6412e-01],
        [-1.3331e-01,  2.0073e-01,  1.3766e-01],
        [ 4.1188e-02,  3.6316e-01, -4.0002e-01],
        [ 1.1602e-01, -2.2928e-01, -8.4232e-02],
        [ 3.6628e-01,  2.1939e-03, -7.7200e-02],
        [-9.3783e-02,  4.2242e-01, -8.1898e-02],
        [ 2.8314e-01,  3.9384e-01, -5.4406e-02],
        [ 2.4333e-01, -6.2231e-02,  5.0183e-02],
        [ 8.3639e-02,  5.0924e-02, -3.3784e-03],
        [ 3.6789e-01, -9.4257e-02, -3.1544e-01],
        [ 2.6537e-01, -2.2608e-01,  3.0173e-01],
        [ 4.1376e-01, -2.0723e-02, -9.7583e-02],
        [ 1.6983e-01, -1.7244e-01, -8.5138e-02],
        [-3.8402e-02, -7.5647e-02,  1.7384e-01],
        [ 2.5068e-01,  1.9365e-01, -2.4366e-01],
        [ 2.1272e-01,  6.6451e-02,  2.3712e-01],
        [-1.4738e-01, -1.5499e-01, -1.8508e-01],
        [-2.6014e-01,  7.0196e-04, -3.5519e-01],
        [ 1.4630e-02

(Epoch 1) TRAIN LOSS:1.1244 LR:0.00000300:   8%|███▎                                     | 2/25 [00:00<00:08,  2.79it/s]

SequenceClassifierOutput(loss=tensor(1.0610, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 3.3358e-01,  2.7102e-02,  1.7650e-01],
        [-1.6536e-01, -2.9028e-01, -1.5298e-01],
        [-1.3705e-01, -3.3812e-01,  1.9219e-01],
        [ 8.6969e-02,  5.7553e-02, -2.3976e-01],
        [-2.4553e-01, -1.2515e-04, -1.9184e-01],
        [-2.2806e-01, -1.2971e-01,  1.4047e-01],
        [-4.4066e-01,  2.6273e-02,  6.6142e-02],
        [ 5.9680e-02, -1.1481e-01,  7.5484e-02],
        [-1.4658e-01,  9.3736e-02, -1.7792e-01],
        [ 2.8832e-02, -1.0281e-01,  3.6410e-01],
        [ 2.2029e-01,  6.0928e-02,  7.9498e-02],
        [ 2.3717e-01, -1.0097e-01, -4.8112e-01],
        [ 9.6440e-02, -1.7729e-01,  7.1182e-02],
        [-9.4717e-03, -2.3322e-02, -1.3896e-01],
        [ 3.6283e-01, -2.9003e-01, -6.5457e-03],
        [-2.1048e-01, -3.0933e-02, -7.2701e-02],
        [ 3.3075e-01, -5.3597e-02,  5.7673e-02],
        [ 9.8947e-02, -6.1667e-02, -1.4133e-01],
        [ 3.1645e-01

(Epoch 1) TRAIN LOSS:1.1033 LR:0.00000300:  12%|████▉                                    | 3/25 [00:01<00:08,  2.61it/s]

...

SequenceClassifierOutput(loss=tensor(0.4035, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.5815, -1.7565,  2.8998],
        [-0.1026, -1.3515,  2.1387],
        [-0.2231, -1.6775,  2.3368],
        [ 1.3565, -0.1997, -0.2362],
        [ 0.5401, -0.3834,  0.3302],
        [ 0.0814, -1.2113,  1.8388],
        [-0.2713, -1.4638,  2.2241],
        [-0.3238, -1.3101,  2.4380],
        [-0.2755, -1.4247,  2.0695],
        [-0.0429, -1.5892,  2.4731],
        [ 0.5096, -1.4636,  1.6393],
        [ 0.7692, -0.5971, -0.1007],
        [ 1.0171, -0.3747, -0.0632],
        [-0.2785, -1.5649,  2.4251],
        [ 1.3134, -0.6599, -0.5485],
        [ 0.7682,  0.0579, -0.8875],
        [-0.2617, -1.8513,  2.5391],
        [-0.2929, -1.8388,  2.2553],
        [ 0.6644, -0.2513, -0.2193],
        [-0.7791, -1.5146,  2.6226],
        [ 1.0019,  0.1375, -0.5276],
        [ 1.0932, -0.2564, -0.4420],
        [-0.4276, -1.6528,  2.8680],
        [ 0.3293,  0.4427, -0.6958],
        [-0.26

(Epoch 10) TRAIN LOSS:0.5323 LR:0.00000300:  96%|█████████████████████████████████████▍ | 24/25 [00:05<00:00,  4.38it/s]

SequenceClassifierOutput(loss=tensor(0.3319, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[ 0.7370,  0.0128, -0.6244],
        [-0.1166, -1.3575,  1.6745],
        [-0.8515, -1.4502,  2.7131],
        [-0.5716, -1.5921,  2.4546],
        [ 0.5327,  0.2404,  0.3993],
        [-0.3798, -1.3231,  2.6679],
        [ 0.1771, -0.1609, -0.3348],
        [ 0.1630, -1.2608,  1.7416],
        [ 0.9299, -1.1613,  0.4822],
        [-0.8081, -1.3478,  2.3312],
        [-0.1985, -1.5164,  1.9518],
        [-0.3088, -1.4026,  1.9062],
        [ 1.2152, -0.5553, -0.6626],
        [ 0.5356, -0.6048,  0.5450],
        [-0.6771, -1.8147,  2.9505],
        [-0.5173, -1.2673,  2.5818],
        [ 1.1437, -0.7188, -0.2206],
        [ 0.0199, -1.4023,  1.6556],
        [ 0.8118, -0.5193, -0.3582],
        [-0.3664, -1.5383,  2.4392],
        [-0.2068, -1.9064,  2.4537],
        [-0.3996, -1.5846,  2.3852],
        [ 1.0358, -0.0620, -0.7977],
        [-0.4287, -1.6363,  2.5055],
        [-0.07

(Epoch 10) TRAIN LOSS:0.5243 LR:0.00000300: 100%|███████████████████████████████████████| 25/25 [00:05<00:00,  4.38it/s]

(Epoch 10) TRAIN LOSS:0.5243 ACC:0.78 F1:0.58 REC:0.59 PRE:0.63 LR:0.00000300


In [17]:
# Evaluate on validation
model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm.tqdm(iter(valid_loader), leave=True, total=len(valid_loader))
for i, batch_data in enumerate(pbar):
    batch_seq = batch_data[-1]
    loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
    
    # Calculate total loss
    valid_loss = loss.item()
    total_loss = total_loss + valid_loss

    # Calculate evaluation metrics
    list_hyp += batch_hyp
    list_label += batch_label
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)

    pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
    
metrics = document_sentiment_metrics_fn(list_hyp, list_label)
print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
    total_loss/(i+1), metrics_to_string(metrics)))

VALID LOSS:0.6434 ACC:0.77 F1:0.70 REC:0.69 PRE:0.72:  29%|████████▊                      | 2/7 [00:00<00:00,  6.71it/s]

SequenceClassifierOutput(loss=tensor(0.7156, device='cuda:0'), logits=tensor([[ 0.9848, -1.1901,  0.6303],
        [ 0.9267,  0.1584, -0.4856],
        [ 0.0654, -1.5826,  2.0053],
        [-0.1290, -1.5016,  2.1812],
        [-0.2285, -1.6376,  2.4970],
        [ 0.7982,  0.3852, -1.0152],
        [ 0.3923,  0.3987, -0.4227],
        [ 0.3894,  0.2551, -0.6009],
        [ 0.0919, -1.2441,  1.1212],
        [ 0.5897,  0.5582, -0.8677],
        [ 1.2826, -0.1770, -1.1496],
        [-0.1299, -0.3111,  0.5171],
        [ 0.2921,  0.3444, -0.7956],
        [ 0.6041,  0.5338, -1.0841],
        [ 0.3990, -0.4024, -0.0963],
        [ 0.7475, -0.6365,  0.4139],
        [ 0.4781, -0.8977,  1.1694],
        [-0.3028, -1.4150,  2.6588],
        [ 0.1691,  0.3640, -0.7206],
        [ 0.8735, -0.3155, -0.5683],
        [ 0.5605,  0.8164, -1.5068],
        [ 0.8352, -1.0822,  0.3040],
        [-0.1292, -1.6275,  2.5480],
        [ 0.2016,  0.1394, -0.6559],
        [ 0.2465, -0.0344, -0.3429],
     

VALID LOSS:0.5922 ACC:0.77 F1:0.67 REC:0.66 PRE:0.70:  57%|█████████████████▋             | 4/7 [00:00<00:00,  6.56it/s]

SequenceClassifierOutput(loss=tensor(0.4090, device='cuda:0'), logits=tensor([[ 0.0661,  0.2467, -0.3563],
        [-0.6512, -1.7945,  2.7131],
        [ 1.3169, -0.9718, -0.3663],
        [ 0.5506, -0.9534,  0.8528],
        [-0.4600, -1.4777,  2.3229],
        [ 1.1033, -0.9344,  0.4311],
        [-0.9087, -1.4477,  2.5952],
        [-0.7602, -1.5999,  2.7431],
        [ 0.0925,  0.5303, -0.6430],
        [ 1.0660, -0.5138, -0.0588],
        [-0.5783, -1.7088,  2.6895],
        [-0.6451, -1.4130,  2.5237],
        [ 0.1043, -1.2336,  1.4781],
        [ 1.3178, -0.9273, -0.0950],
        [-0.7764, -1.3475,  2.3681],
        [-0.2857, -1.7744,  2.4125],
        [-0.5193, -1.7904,  2.7724],
        [-0.5416, -1.5884,  2.3327],
        [-0.6641, -1.5208,  2.5290],
        [ 0.8212, -1.1392,  1.0042],
        [-0.9900, -1.7187,  2.7551],
        [-0.4491, -1.5059,  2.3621],
        [ 0.3366,  0.2360, -0.6432],
        [ 1.2116, -1.0346,  0.0141],
        [ 1.2889, -0.1418, -0.9872],
     

VALID LOSS:0.5492 ACC:0.79 F1:0.70 REC:0.68 PRE:0.72:  86%|██████████████████████████▌    | 6/7 [00:00<00:00,  6.97it/s]

SequenceClassifierOutput(loss=tensor(0.3296, device='cuda:0'), logits=tensor([[ 1.3106, -0.4925, -0.8903],
        [ 1.4060, -0.2686, -0.2209],
        [-0.7131, -1.5191,  2.6861],
        [-0.0631, -1.5453,  1.9726],
        [-0.6472, -1.3325,  2.3653],
        [-0.1313, -1.4732,  2.1454],
        [ 1.2802, -0.1038, -0.8897],
        [-0.3357, -1.1656,  1.9239],
        [-0.3917, -1.0866,  1.8739],
        [ 1.3551, -0.2239, -0.9130],
        [ 0.8051, -0.8648, -0.4330],
        [-0.4231, -1.4522,  2.1662],
        [-0.3290, -1.7561,  2.4750],
        [ 0.1985, -1.0920,  0.9990],
        [ 0.9586, -0.1668, -0.8865],
        [ 0.9274, -1.2588,  0.5371],
        [ 0.7759, -0.7079,  0.4429],
        [-0.3891, -1.4798,  2.7241],
        [-0.0632, -1.1153,  2.0037],
        [ 0.2904,  0.4111, -0.0504],
        [-0.3072, -1.7968,  2.5616],
        [ 1.0335, -1.2880,  0.5263],
        [-0.0695, -1.7118,  2.0791],
        [ 0.3459,  0.1229, -0.9239],
        [ 1.1111, -1.2679,  0.4993],
     

VALID LOSS:0.5617 ACC:0.79 F1:0.70 REC:0.69 PRE:0.72: 100%|███████████████████████████████| 7/7 [00:01<00:00,  6.37it/s]

(Epoch 10) VALID LOSS:0.5617 ACC:0.79 F1:0.70 REC:0.69 PRE:0.72
